## Part 1: Preprocessing

In [21]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [22]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [23]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [24]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    'Age', 'WorkLifeBalance', 'DistanceFromHome', 'Education',
    'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement',
    'JobLevel', 'JobSatisfaction', 'TotalWorkingYears'
]

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes



,0
Age,int64
WorkLifeBalance,int64
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
HourlyRate,int64
JobInvolvement,int64
JobLevel,int64
JobSatisfaction,int64
TotalWorkingYears,int64


In [25]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [26]:
# Convert your X data to numeric data types however you see fit

# Convert categorical columns to numeric using one-hot encoding
X_numeric = pd.get_dummies(X_df)

# Display the first few rows of the transformed data
X_numeric.head()


,Age,WorkLifeBalance,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,TotalWorkingYears
0,41,1,1,2,2,94,3,2,4,8
1,49,3,8,1,3,61,2,2,2,10
2,37,3,2,2,4,92,2,1,3,7
3,33,3,3,4,4,56,3,1,3,8
4,27,3,2,1,1,40,3,1,2,6


In [27]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [29]:

# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded = encoder.transform(y_train[['Department']])
y_test_encoded = encoder.transform(y_test[['Department']])




In [31]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])



## Create, Compile, and Train the Model

In [32]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)


In [33]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(y_train_encoded.shape[1], activation='softmax', name='department_output')(department_hidden_layer)



In [34]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition_encoded.shape[1], activation='softmax', name='attrition_output')(attrition_hidden_layer)



In [35]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             51 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             34 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,925 (15.33 KB)

 Trainable params: 3,925 (15.33 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# Train the model
# Train the model
history = model.fit(
    X_train_scaled,
    {'department_output': y_train_encoded, 'attrition_output': y_train_attrition_encoded},
    epochs=100,
    batch_size=32,
    validation_data=(X_test_scaled, {'department_output': y_test_encoded, 'attrition_output': y_test_attrition_encoded}),
    verbose=1
)


Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - attrition_output_accuracy: 0.8159 - department_output_accuracy: 0.3007 - loss: 1.6461 - val_attrition_output_accuracy: 0.8673 - val_department_output_accuracy: 0.4762 - val_loss: 1.3121
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8234 - department_output_accuracy: 0.5813 - loss: 1.3215 - val_attrition_output_accuracy: 0.8673 - val_department_output_accuracy: 0.6667 - val_loss: 1.1546
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - attrition_output_accuracy: 0.8338 - department_output_accuracy: 0.6528 - loss: 1.1993 - val_attrition_output_accuracy: 0.8673 - val_department_output_accuracy: 0.6667 - val_loss: 1.1422
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - attrition_output_accuracy: 0.8291 - department_output_accuracy: 0.6401 - loss: 1.2112 - val_attrition_output_accuracy: 0.8673 - val_department_output_accuracy: 0.6667 - val_loss: 1.1349
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [37]:
# Evaluate the model with the testing data
evaluation = model.evaluate(
    X_test_scaled,
    {'department_output': y_test_encoded, 'attrition_output': y_test_attrition_encoded},
    verbose=1
)

# Print the evaluation list to understand its structure
print(evaluation)

# Extract the specific metrics
test_loss = evaluation[0]
department_output_loss = evaluation[1]
attrition_output_loss = evaluation[2]

print(f"Test Loss: {test_loss}")
print(f"Test Department Output Loss: {department_output_loss}")
print(f"Test Attrition Output Loss: {attrition_output_loss}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8004 - department_output_accuracy: 0.5711 - loss: 2.0722 
[1.889302134513855, 0.8129251599311829, 0.6020408272743225]
Test Loss: 1.889302134513855
Test Department Output Loss: 0.8129251599311829
Test Attrition Output Loss: 0.6020408272743225


In [38]:
# Print the accuracy for both department and attrition

# Calculate accuracy for the department output
department_pred = model.predict(X_test_scaled)[0]
department_accuracy = np.mean(np.argmax(department_pred, axis=1) == np.argmax(y_test_encoded, axis=1))

# Calculate accuracy for the attrition output
attrition_pred = model.predict(X_test_scaled)[1]
attrition_accuracy = np.mean(np.argmax(attrition_pred, axis=1) == np.argmax(y_test_attrition_encoded, axis=1))

print(f"Test Department Output Accuracy: {department_accuracy}")
print(f"Test Attrition Output Accuracy: {attrition_accuracy}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Test Department Output Accuracy: 0.6020408163265306
Test Attrition Output Accuracy: 0.8129251700680272


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric due to potential class imbalance, which can skew results by favoring the majority class. Metrics like precision, recall, and F1 score offer a more accurate assessment of model performance, especially for minority classes.
2. For the output layers, a common choice would be:
- **For binary classification (attrition)**: The **sigmoid** activation function is typically used, as it outputs values between 0 and 1, making it suitable for binary outcomes.
- **For multi-class classification (department)**: The **softmax** activation function is often chosen, as it outputs a probability distribution across multiple classes, allowing the model to indicate the most likely department.
These choices help ensure that the outputs are interpretable as probabilities, which is important for decision-making.
3. To improve the model, consider several strategies:
- **Feature Engineering**: Add or refine features to better capture relationships in the data.
- **Hyperparameter Tuning**: Experiment with learning rates, batch sizes, and network architectures for optimal performance.
- **Regularization**: Use dropout or L2 regularization to mitigate overfitting.
- **Cross-Validation**: Implement k-fold cross-validation to ensure robustness.
- **Ensemble Methods**: Combine predictions from multiple models to enhance accuracy.
- **Class Imbalance Handling**: Use techniques like oversampling, undersampling, or class weights to address class imbalances.